In [1]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import warnings  

In [2]:
from dataprocess import dataprocessor as dp
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame

with warnings.catch_warnings():  
    warnings.filterwarnings("ignore",category=FutureWarning)
    from keras.models import load_model
    from nn_source import models as mp
    from rl_source import alumnienv

Using TensorFlow backend.


In [3]:
# read the pickled file for ahu data
dfdata = dp.readfile('../data/processed/buildingdata.pkl')

# return pickled df
df = dfdata.return_df(processmethods=['file2df'])

In [4]:
# 0-1 Scale the dataframe before sending it to the environment
scaler = MinMaxScaler(feature_range=(0,1))
dfscaler = scaler.fit(df)

In [5]:
dfscaled = DataFrame(dfscaler.fit_transform(df), index=df.index, columns=df.columns)

In [7]:
energymodel = load_model('../results/lstmtrain/LSTM_model_44_0.01')  # Specify the path where energy is stored

In [8]:
# important parameters to scale the reward
params = {}
param_keys = [
    'energy_saved', 'energy_savings_thresh', 'energy_penalty', 'energy_reward_weight', 
    'comfort', 'comfort_thresh', 'uncomfortable', 'comfort_reward_weight',
    'action_minmax']
param_values = [
    1.0, 0.00, -1.0, 1.0,
    1.0, 0.10, -1.0, 1.0,
    [np.array([df.sat.min()]),np.array([df.sat.max()])]
               ]
for i,j in zip(param_keys,param_values):
    params[i] = j

# instantiate the environment
envmodel = alumnienv.Env(
    df=dfscaled,
    obs_space_vars=['oat', 'orh', 'ghi', 'sat', 'avg_stpt'],  # state space variable
    action_space_vars=['sat'],   # action space variable
    action_space_bounds=[[-2.0], [2.0]],  # bounds for real world action space; is scaled internally using the params
    energy_model=energymodel,  # trained lstm model
    model_input_shape=(1, 1, 4),  # lstm model input data shape (no_samples, output_timestep, inputdim)
    model_input_vars=['oat', 'orh', 'ghi','sat'],  # lstm model input variables
    **params
)

/data/nauga/SmartBuildings/venvs/sbvenv1/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


### Test the environment

In [9]:
# make sure environment is in train mode
envmodel.trainenv()

In [10]:
# resest environment
envmodel.reset()

array([0.79991203, 0.82282514, 0.58238342, 0.7513201 , 0.45607235])

In [11]:
# Step through the environment eg apply a change of 0.43*F in the up direction
envmodel.step(np.array([0.43]))

(array([0.79991203, 0.82282514, 0.60103627, 0.76295517, 0.45607235]),
 0.0,
 False,
 {})

In [12]:
# make sure environment is in test mode
envmodel.testenv()

In [13]:
# resest environment
envmodel.reset()

array([0.73546375, 0.47605578, 0.        , 0.51102325, 0.57364341])

In [14]:
# Step through the environment eg apply a change of 0.43*F in the down direction
envmodel.step(np.array([-0.43]))

(array([0.73546375, 0.47954249, 0.        , 0.49938818, 0.57364341]),
 0.0,
 False,
 {'time': '2019-09-26 01:50:00',
  'energy': 0.322563499212265,
  'baseline_energy': 0.32253530621528625,
  'reward_energy': -1.0,
  'reward_comfort': 1.0,
  'oat': 0.7354637489469744,
  'orh': 0.47605577951877737,
  'T_rl_disch': 0.49938817625757426,
  'avg_stpt': 0.5736434108527124})